In [35]:
# system imports
import os
import sys

# data science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import seaborn as sns

# signal processing
from scipy import signal
from scipy.ndimage import label
from scipy.stats import zscore
from scipy.interpolate import interp1d
from scipy.integrate import trapz


# misc
import warnings

import glob

##
import pytz
import datetime as dt
import math
import seaborn as sns

import pickle

In [36]:

#style settings
sns.set(style='whitegrid', rc={'axes.facecolor': 'white'})

#sns.set_style({'font.family':'Arial', 'font.serif':'Times New Roman'})

## Time Domain analysis

In [37]:
def timedomain(rr):
    results = {}

    hr = 60000/rr
    
    results['Mean RR (ms)'] = np.mean(rr)
    results['STD RR/SDNN (ms)'] = np.std(rr)
    results['Mean HR (Kubios\' style) (beats/min)'] = 60000/np.mean(rr)
    results['Mean HR (beats/min)'] = np.mean(hr)
    results['STD HR (beats/min)'] = np.std(hr)
    results['Min HR (beats/min)'] = np.min(hr)
    results['Max HR (beats/min)'] = np.max(hr)
    results['RMSSD (ms)'] = np.sqrt(np.mean(np.square(np.diff(rr))))
    results['NNxx'] = np.sum(np.abs(np.diff(rr)) > 50)*1
    results['pNNxx (%)'] = 100 * np.sum((np.abs(np.diff(rr)) > 50)*1) / len(rr)
    return results

## Related functions

In [126]:
def get_non_nan_values_cols(p_df, col_name):
    
    slic_ind=p_df[col_name].isnull()
    list_ne=slic_ind.tolist()
    false_indices = [i for i in range(len(list_ne)) if not list_ne[i]]
    
    list_col_interest=p_df[col_name].to_list()
    
    non_values_list = []
    
    for false_ind in false_indices:
        non_val = list_col_interest[false_ind]
        non_values_list.append(non_val)
    
    return non_values_list


def get_file_names_from_ind_list(files_list, inds_list):
    
    selected_files_list=[files_list[sel_index] for sel_index in inds_list]
    
    return selected_files_list


def from_rr_to_time_freq_anal(sk_hr_files_list, condition_files):

    #mean_sk_acc=[]
    #std_sk_acc = []

    #mean_hr_acc=[]
    #std_hr_acc=[]

    for selected_path in sk_hr_files_list:
        dir_path, file_name=os.path.split(selected_path)
        sub_id = file_name[0:12]
        #print(sub_id)
        for cond_file in condition_files:
            if sub_id==cond_file:
                #print(selected_path)
                
                with open(selected_path, 'rb') as f:
                    ibi_data = pickle.load(f)
                    
                #sk_hr_df = pd.read_csv(selected_path)
                
                subject_id = ibi_data["subject_id"]
                event_interst = ibi_data["event_interst"]
                
                all_rr_one_list = ibi_data["all_rr_one_list"]
                
                all_rr_sublists = ibi_data["all_rr_sublists"]
                
                #max_rr = np.max(all_rr_one_list)
                #print(max_rr)
                #min_rr = np.min(all_rr_one_list)
                #print(min_rr)
                
                #range_rr = max_rr - min_rr
                
                
                for index, selected_event in enumerate(event_interst):
                    
                    
                    print(selected_event)
                    sel_rr = all_rr_sublists[index]
                    hr_from_rr = 60000/sel_rr
                    
                    print(np.mean(hr_from_rr))
                    
                    #print(60000/sel_rr)
                    #min_sub = sel_rr - min_rr
                    #scaled_data = min_sub/range_rr
                    
                    #print(scaled_data)
                    
                    
                    #print(results)
                    
                    
                
                
                
                
                
                

                #mean_sk = sk_hr_df[mean_sk_col].values
                #std_sk = sk_hr_df[std_sk_col].values
                #mean_hr =sk_hr_df[mean_hr_col].values
                #std_hr =sk_hr_df[std_hr_col].values


                #mean_sk_acc.append(mean_sk)
                #std_sk_acc.append(std_sk)
                #mean_hr_acc.append(mean_hr)
                #std_hr_acc.append(std_hr)



    #mean_mean_sk=np.mean(np.array(mean_sk_acc), axis=0) 
    #std_mean_sk = np.std(np.array(std_sk_acc), axis=0)
    #std_mean_sk = np.mean(np.array(std_sk_acc), axis=0)

    #mean_mean_hr=np.mean(np.array(mean_hr_acc), axis=0) 
    #std_mean_hr = np.std(np.array(std_hr_acc), axis=0)
    #std_mean_hr = np.mean(np.array(std_hr_acc), axis=0)
    
    #selected_events = sk_hr_df[event_interst_col].to_list()
    
    #extracted_data={"selected_event": selected_events, "hr_mean": mean_mean_hr, "hr_std": std_mean_hr, "sk_mean": mean_mean_sk, "sk_std":std_mean_sk}
    
    #return extracted_data


In [127]:
sub_data_path = "/home/muhammad/Desktop/Datasets/data_sony_digiRelax/subjective_data/subjective_data.xlsx"
#hrv_data_dir = "/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp3/results_22sub/hrv_analysis"
hrv_data_dir = "/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_all_sub/hrv/ibi_pkl"


In [128]:
sk_hr_files_list = glob.glob(hrv_data_dir+ "/*.pkl")

In [129]:
sub_data_df=pd.read_excel(sub_data_path)
file_col = "subject"
cond_col ="condition"

file_list = get_non_nan_values_cols(sub_data_df,file_col)
cond_list = get_non_nan_values_cols(sub_data_df, cond_col)

cond_set=set(cond_list)
unique_conds_list= list(cond_set)

In [130]:
print(unique_conds_list)

['2D_scent', 'VR_scent', 'VR_no_scent', '2D_no_scent']


In [131]:
file_list

['VP004_081123',
 'VP005_081123',
 'VP006_091123',
 'VP007_091123',
 'VP008_101123',
 'VP009_101123',
 'VP010_131123',
 'VP011_131123',
 'VP012_141123',
 'VP013_151123',
 'VP014_151123',
 'VP015_161123',
 'VP016_161123',
 'VP017_171123',
 'VP018_201123',
 'VP019_201123',
 'VP020_221123',
 'VP021_221123',
 'VP022_231123']

In [132]:
no_scent_2d_inds=np.where(np.array(cond_list)=="2D_no_scent")
no_scent_2d_inds = list(no_scent_2d_inds[0])
no_scent_2d_files =get_file_names_from_ind_list(file_list, no_scent_2d_inds)

scent_2d_inds=np.where(np.array(cond_list)=="2D_scent")
scent_2d_inds=list(scent_2d_inds[0])
scent_2d_files=get_file_names_from_ind_list(file_list, scent_2d_inds)

no_scent_vr_inds = np.where(np.array(cond_list)=="VR_no_scent")
no_scent_vr_inds=list(no_scent_vr_inds[0])
no_scent_vr_files=get_file_names_from_ind_list(file_list, no_scent_vr_inds)

scent_vr_inds =np.where(np.array(cond_list)=="VR_scent")
scent_vr_inds =list(scent_vr_inds[0])
scent_vr_files=get_file_names_from_ind_list(file_list, scent_vr_inds)

In [133]:
scent_vr_files[0]

'VP006_091123'

In [134]:
from_rr_to_time_freq_anal(sk_hr_files_list, scent_vr_files)

baseline_instruction_ts 3 min
94.11043362727311
tsst_prep_ts 5 min
99.33809598385477
tsst_pres_ts 5 min
107.35707432519793
tsst_pres_ts 10 min
99.26596391583186
relaxation_prep_ts 5 min
87.3354526357195
relaxation_prep_ts 10 min
87.2434463585723
relaxation_prep_ts 15 min
88.78520560809919
baseline_instruction_ts 3 min
68.52684204915856
tsst_prep_ts 5 min
91.81505088366143
tsst_pres_ts 5 min
92.23932639860115
tsst_pres_ts 10 min
84.51399928697774
relaxation_prep_ts 5 min
72.21675634146874
relaxation_prep_ts 10 min
72.0129618109361
relaxation_prep_ts 15 min
72.057100024432
baseline_instruction_ts 3 min
48.86731941634104
tsst_prep_ts 5 min
55.50326253346148
tsst_pres_ts 5 min
60.335698027390826
tsst_pres_ts 10 min
63.55125856017843
relaxation_prep_ts 5 min
48.2515937371986
relaxation_prep_ts 10 min
49.914431200676795
relaxation_prep_ts 15 min
49.82299309763204
baseline_instruction_ts 3 min
81.60859747816652
tsst_prep_ts 5 min
109.01565057705714
tsst_pres_ts 5 min
122.23623808420884
tsst_p

In [105]:
scent_vr_files

['VP006_091123',
 'VP007_091123',
 'VP010_131123',
 'VP011_131123',
 'VP017_171123']

In [106]:
no_scent_vr_files


['VP008_101123',
 'VP009_101123',
 'VP013_151123',
 'VP014_151123',
 'VP018_201123',
 'VP019_201123',
 'VP020_221123',
 'VP021_221123']

In [107]:
scent_2d_files

['VP004_081123', 'VP005_081123', 'VP022_231123']